In [72]:
import keras
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils
from keras.layers import Dense, Dropout, LSTM, Embedding, Reshape, Input, InputLayer, Concatenate, Lambda, Add
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential, Model
from keras import backend as K
from keras.preprocessing.text import Tokenizer
import random

In [27]:
df = pd.read_csv('haikus_with_syllables.csv', encoding='latin1')
df = df[df['lang'] == 'en'].copy()
df = df[~pd.isnull(df['0']) & ~pd.isnull(df['1']) & ~pd.isnull(df['2'])].copy()
df = df[~pd.isnull(df['0_syllables']) & ~pd.isnull(df['1_syllables']) & ~pd.isnull(df['2_syllables'])].copy()

# Duplicate lines with ambiguous syllable counts
lines = set([0, 1, 2])

for i in range(3):
    lines.remove(i)
    df = df[[
        '0', '1', '2',
        #'1_syllables', '2_syllables'
    ] + ['%s_syllables' % j for j in lines]].join(
        df['%s_syllables' % i].str.split(
            ',', expand=True
        ).stack(-1).reset_index(
            level=1, drop=True
        ).rename('%s_syllables' % i)
    ).drop_duplicates()
    lines.add(i)

df

,0,1,2,0_syllables,1_syllables,2_syllables
0,Memorial Day --,a shadow for each,white cross,5,5,2
1,spring rain -,as the doctor speaks,i think of lilacs,2,5,5
1,spring rain -,as the doctor speaks,i think of lilacs,3,5,5
2,spring moonset --,a rice ball for,breakfast,3,4,2
2,spring moonset --,a rice ball for,breakfast,4,4,2
3,sunny afternoon,an old man lingers,near the mailbox,5,5,4
4,cinco de mayo,horses roll,in the shallows,5,3,4
5,quitting time,the smell of rain,in the lobby,3,4,4
6,waves,slowly cresting towards shore,a faint moon,1,6,3
6,waves,slowly cresting towards shore,a faint moon,1,7,3


In [28]:
max_line_len = max([max(df['%s' % i].str.len()) for i in range(3)])

for i in range(3):
    i = str(i)
    df['%s_in' % i] = df[i].str[0] + df[i].str.pad(max_line_len, 'right', '\n')
    df['%s_out' % i] = df[i].str.pad(max_line_len, 'right', '\n') + '\n'
    
max_line_len += 1

df

,0,1,2,0_syllables,1_syllables,2_syllables,0_in,0_out,1_in,1_out,2_in,2_out
0,Memorial Day --,a shadow for each,white cross,5,5,2,MMemorial Day --\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,Memorial Day --\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\...,aa shadow for each\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,a shadow for each\n\n\n\n\n\n\n\n\n\n\n\n\n\n\...,wwhite cross\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,white cross\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\...
1,spring rain -,as the doctor speaks,i think of lilacs,2,5,5,sspring rain -\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,spring rain -\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\...,aas the doctor speaks\n\n\n\n\n\n\n\n\n\n\n\n\...,as the doctor speaks\n\n\n\n\n\n\n\n\n\n\n\n\n...,ii think of lilacs\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,i think of lilacs\n\n\n\n\n\n\n\n\n\n\n\n\n\n\...
1,spring rain -,as the doctor speaks,i think of lilacs,3,5,5,sspring rain -\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,spring rain -\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\...,aas the doctor speaks\n\n\n\n\n\n\n\n\n\n\n\n\...,as the doctor speaks\n\n\n\n\n\n\n\n\n\n\n\n\n...,ii think of lilacs\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,i think of lilacs\n\n\n\n\n\n\n\n\n\n\n\n\n\n\...
2,spring moonset --,a rice ball for,breakfast,3,4,2,sspring moonset --\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,spring moonset --\n\n\n\n\n\n\n\n\n\n\n\n\n\n\...,aa rice ball for\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,a rice ball for\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\...,bbreakfast\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,breakfast\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\...
2,spring moonset --,a rice ball for,breakfast,4,4,2,sspring moonset --\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,spring moonset --\n\n\n\n\n\n\n\n\n\n\n\n\n\n\...,aa rice ball for\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,a rice ball for\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\...,bbreakfast\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,breakfast\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\...
3,sunny afternoon,an old man lingers,near the mailbox,5,5,4,ssunny afternoon\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,sunny afternoon\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\...,aan old man lingers\n\n\n\n\n\n\n\n\n\n\n\n\n\...,an old man lingers\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,nnear the mailbox\n\n\n\n\n\n\n\n\n\n\n\n\n\n\...,near the mailbox\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...
4,cinco de mayo,horses roll,in the shallows,5,3,4,ccinco de mayo\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,cinco de mayo\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\...,hhorses roll\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,horses roll\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\...,iin the shallows\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,in the shallows\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\...
5,quitting time,the smell of rain,in the lobby,3,4,4,qquitting time\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,quitting time\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\...,tthe smell of rain\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,the smell of rain\n\n\n\n\n\n\n\n\n\n\n\n\n\n\...,iin the lobby\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\...,in the lobby\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...
6,waves,slowly cresting towards shore,a faint moon,1,6,3,wwaves\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,waves\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\...,sslowly cresting towards shore\n\n\n\n\n\n\n\n...,slowly cresting towards shore\n\n\n\n\n\n\n\n\...,aa faint moon\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\...,a faint moon\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...
6,waves,slowly cresting towards shore,a faint moon,1,7,3,wwaves\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,waves\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\...,sslowly cresting towards shore\n\n\n\n\n\n\n\n...,slowly cresting towards shore\n\n\n\n\n\n\n\n\...,aa faint moon\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\...,a faint moon\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...


In [34]:
inputs = df[['0_in', '1_in', '2_in']].values

t = Tokenizer(filters='', char_level=True)
t.fit_on_texts(inputs.flatten())
X = np_utils.to_categorical([
    t.texts_to_sequences(inputs[:,i]) for i in range(3)
])

outputs = df[['0_out', '1_out', '2_out']].values

#t = Tokenizer(filters='', char_level=True)
#t.fit_on_texts(outputs.flatten())
Y = np_utils.to_categorical([
    t.texts_to_sequences(outputs[:,i]) for i in range(3)
])

n_tokens = len(t.word_counts) + 1

X_syllables = df[['0_syllables', '1_syllables', '2_syllables']].values

In [52]:
latent_dim = 128

# Keras Model

inputs = [Input(shape=(None, n_tokens), name='input_line_%s' % i) for i in range(3)]

syllables = [Input(shape=(1,), name='syllables_line_%s' % i) for i in range(3)]
syllable_denses = [Dense(latent_dim, activation='relu', name='syllable_dense_%s' % i) for i in range(3)]
syllable_dense_outputs = [syllable_denses[i](syllables[i]) for i in range(3)]

lstms = [LSTM(latent_dim, return_state=True, return_sequences=True, name='lstm_%s' % i) for i in range(3)]

lstm_out, lstm_h, lstm_c = [None, None, None], [None, None, None], [None, None, None]
for i in range(3):
    if i > 0:
        lstm_out[i], lstm_h[i], lstm_c[i] = lstms[i](inputs[i], initial_state=[
            Add(name='add_h_%s' % i)([
                lstm_h[i-1],
                syllable_dense_outputs[i]
            ]),
            Add(name='add_c_%s' % i)([
                lstm_c[i-1],
                syllable_dense_outputs[i]
            ])
        ])
    else:
        lstm_out[i], lstm_h[i], lstm_c[i] = lstms[i](inputs[i], initial_state=[
            syllable_dense_outputs[i], syllable_dense_outputs[i]
        ])
        
denses = [Dense(n_tokens, activation='softmax', name='output_%s' % i) for i in range(3)]
outputs = [denses[i](lstm_out[i]) for i in range(3)]

# Enforce number of chars per line
#counter = Lambda(lambda x: K.sum(K.cast(K.less(K.argmax(x, axis=2), 60), K.floatx()), axis=1, keepdims=True))
#counts = [counter(i) for i in outputs]

#model = Model(inputs, outputs + counts)

model = Model(inputs + syllables, outputs)

# Use categorical for the character outputs and MSE for the counts
#model.compile(optimizer='rmsprop', loss=['categorical_crossentropy']*3 + ['mean_squared_error']*3)
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')
model.summary(line_length=200)

________________________________________________________________________________________________________________________________________________________________________________________________________
Layer (type)                                                      Output Shape                                Param #                 Connected to                                                      
syllables_line_0 (InputLayer)                                     (None, 1)                                   0                                                                                         
________________________________________________________________________________________________________________________________________________________________________________________________________
input_line_0 (InputLayer)                                         (None, None, 58)                            0                                                                                     

In [58]:
model.fit([
    X[0], X[1], X[2],
    X_syllables[:,0], X_syllables[:,1], X_syllables[:,2]
], [Y[0], Y[1], Y[2]], batch_size=128, epochs=10)

Epoch 1/10
5502/5502 [==============================] - 10s 2ms/step - loss: 5.0002 - output_0_loss: 1.5830 - output_1_loss: 1.8479 - output_2_loss: 1.5693
Epoch 2/10
5502/5502 [==============================] - 7s 1ms/step - loss: 3.7672 - output_0_loss: 1.1586 - output_1_loss: 1.4415 - output_2_loss: 1.1671
Epoch 3/10
5502/5502 [==============================] - 7s 1ms/step - loss: 3.5969 - output_0_loss: 1.1034 - output_1_loss: 1.3773 - output_2_loss: 1.1161
Epoch 4/10
5502/5502 [==============================] - 7s 1ms/step - loss: 3.4711 - output_0_loss: 1.0717 - output_1_loss: 1.3287 - output_2_loss: 1.0707
Epoch 5/10
5502/5502 [==============================] - 8s 1ms/step - loss: 3.3446 - output_0_loss: 1.0448 - output_1_loss: 1.2661 - output_2_loss: 1.0336
Epoch 6/10
5502/5502 [==============================] - 8s 1ms/step - loss: 3.1936 - output_0_loss: 1.0137 - output_1_loss: 1.1939 - output_2_loss: 0.9860
Epoch 7/10
5502/5502 [==============================] - 8s 2ms/step -

In [59]:
model.fit([
    X[0], X[1], X[2],
    X_syllables[:,0], X_syllables[:,1], X_syllables[:,2]
], [Y[0], Y[1], Y[2]], batch_size=128, epochs=50)

Epoch 1/50
5502/5502 [==============================] - 7s 1ms/step - loss: 2.5966 - output_0_loss: 0.8234 - output_1_loss: 0.9893 - output_2_loss: 0.7838
Epoch 2/50
5502/5502 [==============================] - 8s 1ms/step - loss: 2.5286 - output_0_loss: 0.7959 - output_1_loss: 0.9695 - output_2_loss: 0.7632
Epoch 3/50
5502/5502 [==============================] - 8s 1ms/step - loss: 2.4686 - output_0_loss: 0.7704 - output_1_loss: 0.9529 - output_2_loss: 0.7453
Epoch 4/50
5502/5502 [==============================] - 8s 1ms/step - loss: 2.4191 - output_0_loss: 0.7487 - output_1_loss: 0.9390 - output_2_loss: 0.7313
Epoch 5/50
5502/5502 [==============================] - 8s 1ms/step - loss: 2.3746 - output_0_loss: 0.7308 - output_1_loss: 0.9253 - output_2_loss: 0.7185
Epoch 6/50
5502/5502 [==============================] - 8s 1ms/step - loss: 2.3347 - output_0_loss: 0.7139 - output_1_loss: 0.9135 - output_2_loss: 0.7073
Epoch 7/50
4864/5502 [=========================>....] - ETA: 0s - loss

KeyboardInterrupt: 

In [60]:
model.fit([
    X[0], X[1], X[2],
    X_syllables[:,0], X_syllables[:,1], X_syllables[:,2]
], [Y[0], Y[1], Y[2]], batch_size=128, epochs=200)

Epoch 1/200
5502/5502 [==============================] - 8s 1ms/step - loss: 2.2694 - output_0_loss: 0.6863 - output_1_loss: 0.8937 - output_2_loss: 0.6894
Epoch 2/200
5502/5502 [==============================] - 8s 1ms/step - loss: 2.2390 - output_0_loss: 0.6730 - output_1_loss: 0.8846 - output_2_loss: 0.6814
Epoch 3/200
5502/5502 [==============================] - 8s 1ms/step - loss: 2.2117 - output_0_loss: 0.6621 - output_1_loss: 0.8756 - output_2_loss: 0.6740
Epoch 4/200
5502/5502 [==============================] - 8s 1ms/step - loss: 2.1839 - output_0_loss: 0.6511 - output_1_loss: 0.8670 - output_2_loss: 0.6658
Epoch 5/200
5502/5502 [==============================] - 8s 1ms/step - loss: 2.1592 - output_0_loss: 0.6410 - output_1_loss: 0.8583 - output_2_loss: 0.6599
Epoch 6/200
5502/5502 [==============================] - 8s 1ms/step - loss: 2.1363 - output_0_loss: 0.6319 - output_1_loss: 0.8517 - output_2_loss: 0.6527
Epoch 7/200
5502/5502 [==============================] - 8s 1ms/

5502/5502 [==============================] - 8s 1ms/step - loss: 1.5574 - output_0_loss: 0.4324 - output_1_loss: 0.6503 - output_2_loss: 0.4746
Epoch 54/200
5502/5502 [==============================] - 17s 3ms/step - loss: 1.5508 - output_0_loss: 0.4298 - output_1_loss: 0.6485 - output_2_loss: 0.4725
Epoch 55/200
5502/5502 [==============================] - 8s 1ms/step - loss: 1.5417 - output_0_loss: 0.4271 - output_1_loss: 0.6449 - output_2_loss: 0.4697
Epoch 56/200
5502/5502 [==============================] - 8s 1ms/step - loss: 1.5356 - output_0_loss: 0.4252 - output_1_loss: 0.6430 - output_2_loss: 0.4674
Epoch 57/200
5502/5502 [==============================] - 8s 1ms/step - loss: 1.5278 - output_0_loss: 0.4224 - output_1_loss: 0.6405 - output_2_loss: 0.4649
Epoch 58/200
5502/5502 [==============================] - 8s 1ms/step - loss: 1.5200 - output_0_loss: 0.4205 - output_1_loss: 0.6372 - output_2_loss: 0.4623
Epoch 59/200
5502/5502 [==============================] - 8s 1ms/step 

5502/5502 [==============================] - 8s 1ms/step - loss: 1.2466 - output_0_loss: 0.3366 - output_1_loss: 0.5395 - output_2_loss: 0.3705
Epoch 105/200
5502/5502 [==============================] - 8s 1ms/step - loss: 1.2416 - output_0_loss: 0.3349 - output_1_loss: 0.5376 - output_2_loss: 0.3691
Epoch 106/200
5502/5502 [==============================] - 8s 1ms/step - loss: 1.2379 - output_0_loss: 0.3346 - output_1_loss: 0.5358 - output_2_loss: 0.3674
Epoch 107/200
5502/5502 [==============================] - 18s 3ms/step - loss: 1.2317 - output_0_loss: 0.3323 - output_1_loss: 0.5339 - output_2_loss: 0.3656
Epoch 108/200
5502/5502 [==============================] - 8s 1ms/step - loss: 1.2284 - output_0_loss: 0.3312 - output_1_loss: 0.5329 - output_2_loss: 0.3643
Epoch 109/200
5502/5502 [==============================] - 8s 1ms/step - loss: 1.2229 - output_0_loss: 0.3294 - output_1_loss: 0.5307 - output_2_loss: 0.3628
Epoch 110/200
5502/5502 [==============================] - 8s 1ms

KeyboardInterrupt: 

In [ ]:
model.fit([
    X[0], X[1], X[2],
    X_syllables[:,0], X_syllables[:,1], X_syllables[:,2]
], [Y[0], Y[1], Y[2]], batch_size=128, epochs=200)

Epoch 1/200
5502/5502 [==============================] - 12s 2ms/step - loss: 1.1140 - output_0_loss: 0.2983 - output_1_loss: 0.4905 - output_2_loss: 0.3251
Epoch 2/200
5502/5502 [==============================] - 13s 2ms/step - loss: 1.1100 - output_0_loss: 0.2967 - output_1_loss: 0.4900 - output_2_loss: 0.3233
Epoch 3/200
5502/5502 [==============================] - 13s 2ms/step - loss: 1.1064 - output_0_loss: 0.2956 - output_1_loss: 0.4880 - output_2_loss: 0.3228
Epoch 4/200
5502/5502 [==============================] - 13s 2ms/step - loss: 1.1024 - output_0_loss: 0.2948 - output_1_loss: 0.4867 - output_2_loss: 0.3210
Epoch 5/200
5502/5502 [==============================] - 13s 2ms/step - loss: 1.0993 - output_0_loss: 0.2934 - output_1_loss: 0.4859 - output_2_loss: 0.3200
Epoch 6/200
5502/5502 [==============================] - 14s 3ms/step - loss: 1.0951 - output_0_loss: 0.2926 - output_1_loss: 0.4845 - output_2_loss: 0.3181
Epoch 7/200
5502/5502 [==============================] - 1

5502/5502 [==============================] - 15s 3ms/step - loss: 0.9516 - output_0_loss: 0.2517 - output_1_loss: 0.4319 - output_2_loss: 0.2680
Epoch 53/200
5502/5502 [==============================] - 15s 3ms/step - loss: 0.9500 - output_0_loss: 0.2517 - output_1_loss: 0.4308 - output_2_loss: 0.2675
Epoch 54/200
5502/5502 [==============================] - 15s 3ms/step - loss: 0.9463 - output_0_loss: 0.2502 - output_1_loss: 0.4300 - output_2_loss: 0.2661
Epoch 55/200
5502/5502 [==============================] - 15s 3ms/step - loss: 0.9426 - output_0_loss: 0.2493 - output_1_loss: 0.4286 - output_2_loss: 0.2648
Epoch 56/200
5502/5502 [==============================] - 15s 3ms/step - loss: 0.9428 - output_0_loss: 0.2489 - output_1_loss: 0.4290 - output_2_loss: 0.2649
Epoch 57/200
5502/5502 [==============================] - 15s 3ms/step - loss: 0.9374 - output_0_loss: 0.2484 - output_1_loss: 0.4261 - output_2_loss: 0.2629
Epoch 58/200
5502/5502 [==============================] - 15s 3ms

5502/5502 [==============================] - 14s 3ms/step - loss: 0.8418 - output_0_loss: 0.2222 - output_1_loss: 0.3913 - output_2_loss: 0.2283
Epoch 104/200
5502/5502 [==============================] - 15s 3ms/step - loss: 0.8392 - output_0_loss: 0.2215 - output_1_loss: 0.3899 - output_2_loss: 0.2279
Epoch 105/200
5502/5502 [==============================] - 15s 3ms/step - loss: 0.8390 - output_0_loss: 0.2209 - output_1_loss: 0.3899 - output_2_loss: 0.2283
Epoch 106/200
5502/5502 [==============================] - 15s 3ms/step - loss: 0.8366 - output_0_loss: 0.2207 - output_1_loss: 0.3885 - output_2_loss: 0.2274
Epoch 107/200
5502/5502 [==============================] - 15s 3ms/step - loss: 0.8351 - output_0_loss: 0.2196 - output_1_loss: 0.3878 - output_2_loss: 0.2278
Epoch 108/200
5502/5502 [==============================] - 15s 3ms/step - loss: 0.8322 - output_0_loss: 0.2194 - output_1_loss: 0.3878 - output_2_loss: 0.2250
Epoch 109/200
5502/5502 [==============================] - 1

5502/5502 [==============================] - 15s 3ms/step - loss: 0.7664 - output_0_loss: 0.2029 - output_1_loss: 0.3623 - output_2_loss: 0.2012
Epoch 155/200
5502/5502 [==============================] - 14s 3ms/step - loss: 0.7637 - output_0_loss: 0.2023 - output_1_loss: 0.3620 - output_2_loss: 0.1994
Epoch 156/200
5502/5502 [==============================] - 16s 3ms/step - loss: 0.7623 - output_0_loss: 0.2012 - output_1_loss: 0.3605 - output_2_loss: 0.2007
Epoch 157/200
5502/5502 [==============================] - 15s 3ms/step - loss: 0.7623 - output_0_loss: 0.2020 - output_1_loss: 0.3609 - output_2_loss: 0.1994
Epoch 158/200
5502/5502 [==============================] - 16s 3ms/step - loss: 0.7620 - output_0_loss: 0.2022 - output_1_loss: 0.3606 - output_2_loss: 0.1992
Epoch 159/200
5502/5502 [==============================] - 17s 3ms/step - loss: 0.7588 - output_0_loss: 0.2008 - output_1_loss: 0.3590 - output_2_loss: 0.1990
Epoch 160/200
5502/5502 [==============================] - 1

In [103]:
generator_syllables

[<tf.Tensor 'generator_syllables_line_0_4:0' shape=(?, 1) dtype=float32>,
 <tf.Tensor 'generator_syllables_line_1_4:0' shape=(?, 1) dtype=float32>,
 <tf.Tensor 'generator_syllables_line_2_4:0' shape=(?, 1) dtype=float32>]

In [105]:
generator_syllable_denses[0]

In [107]:
generator_inputs = [Input(shape=(None, n_tokens), name='generator_input_%s' % i) for i in range(3)]
generator_inputs_h = [Input(shape=(latent_dim,), name='generator_in_h_%s' % i) for i in range(3)]
generator_inputs_c = [Input(shape=(latent_dim,), name='generator_in_c_%s' % i) for i in range(3)]
generator_lstms = [LSTM(latent_dim, return_state=True, return_sequences=True, name='generator_lstm_%s' % i) for i in range(3)]
generator_lstm_out, generator_lstm_h, generator_lstm_c = [None, None, None], [None, None, None], [None, None, None]
generator_denses = [Dense(n_tokens, activation='softmax') for i in range(3)]

generator_syllables = [Input(shape=(1,), name='generator_syllables_%s' % i) for i in range(3)]
generator_syllable_denses = [Dense(latent_dim, activation='relu', name='generator_syllable_dense_%s' % i) for i in range(3)]
#syllable_dense_outputs = [syllable_denses[i](syllables[i]) for i in range(3)]

generator_syllable_dense_outputs = []
generator_outputs = []
generator_models = []

for i in range(3):
    generator_syllable_dense_outputs.append(generator_syllable_denses[i](generator_syllables[i]))
    generator_lstm_out[i], generator_lstm_h[i], generator_lstm_c[i] = generator_lstms[i](
        generator_inputs[i], initial_state=[
            Add(name='generator_add_h_%s' % i)([
                generator_inputs_h[i],
                generator_syllable_dense_outputs[i]
            ]),
            Add(name='generator_add_c_%s' % i)([
                generator_inputs_c[i],
                generator_syllable_dense_outputs[i]
            ])
        ]
    )
    generator_outputs.append(generator_denses[i](generator_lstm_out[i]))

    generator_models.append(Model(
        [generator_inputs[i], generator_syllables[i], generator_inputs_h[i], generator_inputs_c[i]],
        [generator_outputs[i], generator_lstm_h[i], generator_lstm_c[i]]
    ))

    generator_syllable_denses[i].set_weights(syllable_denses[i].get_weights())
    generator_lstms[i].set_weights(lstms[i].get_weights())
    generator_denses[i].set_weights(denses[i].get_weights())

'''# Hook up line 0
generator_syllable_dense_outputs = [generator_syllable_denses[0](generator_syllables[0])]
generator_lstm_out[0], generator_lstm_h[0], generator_lstm_c[0] = generator_lstms[0](
    generator_inputs[0], initial_state=[generator_syllable_dense_outputs[0], generator_syllable_dense_outputs[0]]
)
generator_outputs = [generator_denses[0](generator_lstm_out[0])]
generator_models = [Model([generator_inputs[0], generator_syllables[0]], [generator_outputs[0], generator_lstm_h[0], generator_lstm_c[0]])]

# Set weights for line 0
generator_syllable_denses[0].set_weights(syllable_denses[0].get_weights())
generator_lstms[0].set_weights(lstms[0].get_weights())
generator_denses[0].set_weights(denses[0].get_weights())'''

generator_models[0].summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
generator_syllables_0 (InputLay (None, 1)            0                                            
__________________________________________________________________________________________________
generator_in_h_0 (InputLayer)   (None, 128)          0                                            
__________________________________________________________________________________________________
generator_syllable_dense_0 (Den (None, 128)          256         generator_syllables_0[0][0]      
__________________________________________________________________________________________________
generator_in_c_0 (InputLayer)   (None, 128)          0                                            
__________________________________________________________________________________________________
generator_

In [112]:
fgenerator_inputs = [Input(shape=(None, n_tokens), name='fgenerator_input_%s' % i) for i in range(3)]
fgenerator_inputs_h = [Input(shape=(latent_dim,), name='fgenerator_in_h_%s' % i) for i in range(3)]
fgenerator_inputs_c = [Input(shape=(latent_dim,), name='fgenerator_in_c_%s' % i) for i in range(3)]
fgenerator_lstms = [LSTM(latent_dim, return_state=True, return_sequences=True, name='fgenerator_lstm_%s' % i) for i in range(3)]
fgenerator_lstm_out, fgenerator_lstm_h, fgenerator_lstm_c = [None, None, None], [None, None, None], [None, None, None]
fgenerator_denses = [Dense(n_tokens, activation='softmax') for i in range(3)]

#fgenerator_syllables = [Input(shape=(1,), name='fgenerator_syllables_%s' % i) for i in range(3)]
#fgenerator_syllable_denses = [Dense(latent_dim, activation='relu', name='fgenerator_syllable_dense_%s' % i) for i in range(3)]

#fgenerator_syllable_dense_outputs = []
fgenerator_outputs = []
fgenerator_models = []

for i in range(3):
    #fgenerator_syllable_dense_outputs.append(fgenerator_syllable_denses[i](fgenerator_syllables[i]))
    fgenerator_lstm_out[i], fgenerator_lstm_h[i], fgenerator_lstm_c[i] = fgenerator_lstms[i](
        fgenerator_inputs[i], initial_state=[fgenerator_inputs_h[i], fgenerator_inputs_c[i]]
        
    )
    fgenerator_outputs.append(fgenerator_denses[i](fgenerator_lstm_out[i]))

    fgenerator_models.append(Model(
        [fgenerator_inputs[i], fgenerator_inputs_h[i], fgenerator_inputs_c[i]],
        [fgenerator_outputs[i], fgenerator_lstm_h[i], fgenerator_lstm_c[i]]
    ))

    #generator_syllable_denses[i].set_weights(syllable_denses[i].get_weights())
    fgenerator_lstms[i].set_weights(lstms[i].get_weights())
    fgenerator_denses[i].set_weights(denses[i].get_weights())

fgenerator_models[0].summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
fgenerator_input_0 (InputLayer) (None, None, 58)     0                                            
__________________________________________________________________________________________________
fgenerator_in_h_0 (InputLayer)  (None, 128)          0                                            
__________________________________________________________________________________________________
fgenerator_in_c_0 (InputLayer)  (None, 128)          0                                            
__________________________________________________________________________________________________
fgenerator_lstm_0 (LSTM)        [(None, None, 128),  95744       fgenerator_input_0[0][0]         
                                                                 fgenerator_in_h_0[0][0]          
          

In [108]:
np.zeros(latent_dim)

array([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.])

In [123]:
def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

h,c = np.zeros(latent_dim).reshape((1,-1)), np.zeros(latent_dim).reshape((1,-1))
#first = 'maw'
pattern = [5,5,5]
#generator_models[0].reset_states()
#generator_lstms[0].reset_states()
for i,syllables in enumerate(pattern):
    line = [random.randint(2,21)]
    #line = t.texts_to_sequences(first[i])[0]
    #print(line)
    
    ## FIRST CHAR
    def f():
        while True:
            yield [np_utils.to_categorical(line[-1], num_classes=n_tokens).reshape((1,1,-1)), np.array([syllables]), h, c]
    
    character, h, c = generator_models[i].predict_generator(f(), steps=1)    
    line.append(np.argmax(character))
    
    ## ALL THE REST (Don't include syllables, use fgenerator)
    def f():
        while True:
            yield [np_utils.to_categorical(line[-1], num_classes=n_tokens).reshape((1,1,-1)), h, c]
            
    while len(line) < max_line_len:
        character, h, c = fgenerator_models[i].predict_generator(f(), steps=1)
        line.append(np.argmax(character))
        
        
        #line.append(sample(character[0][0], .01))
        #print (np.argmax(character), line[-1])
        
    #print(line[1:line.index(1)])
    #print(line)
    if 1 in line:
        print(t.sequences_to_texts([line[1:line.index(1)]]))
    else:
        print(t.sequences_to_texts([line[1:]]))

['t h e   s o u n d   o f   s h a r e']
['g r a n d d a y s   o n e   h a n d']
['c r o s s e s   t h e   f i r e f l i e l']


In [94]:
t.texts_to_sequences('a')

[[5]]

In [64]:
generator_models

In [67]:
q = lstm_h[0]

In [70]:
q.shape

TensorShape([Dimension(None), Dimension(128)])

In [86]:
syllable_denses[0].get_weights()

[array([[ -8.69315416e-02,   1.58821270e-02,  -1.16562121e-01,
           2.28478432e-01,  -1.04648210e-01,   2.90493727e-01,
          -1.33755192e-01,   8.07484165e-02,  -3.10301711e-03,
           2.48208925e-01,  -7.16679692e-02,   1.17915593e-01,
          -1.43079162e-02,  -1.82965696e-02,   5.31065166e-01,
          -1.42102137e-01,   2.76264817e-01,  -6.38557225e-02,
          -8.52468610e-02,  -5.85604757e-02,   1.50066227e-01,
           3.20856869e-01,  -2.25959588e-02,  -2.15239599e-01,
          -4.38977182e-02,  -2.07218647e-01,   2.59354293e-01,
           2.35327631e-01,  -3.63578945e-02,   1.76864028e-01,
           1.38739824e-01,  -1.98396429e-01,  -8.30564350e-02,
          -1.00843847e-01,  -1.17549300e-02,   1.97735533e-01,
           1.48821458e-01,  -9.78974551e-02,  -3.36539745e-03,
          -9.45647508e-02,   2.35556737e-01,  -2.14834824e-01,
           6.48017004e-02,   1.63509756e-01,  -1.85196742e-01,
           3.87736440e-01,   4.17508706e-02,  -1.674516

In [87]:
syllable_denses[1].get_weights()

[array([[ 0.00670846,  0.06873073, -0.05807698,  0.19585854,  0.13452879,
          0.14311592, -0.07123995,  0.06893329,  0.32478645,  0.12375577,
          0.02606246,  0.1827348 ,  0.19536008, -0.12195995,  0.22301048,
         -0.0332574 , -0.21234776,  0.25735095, -0.03772701,  0.18757936,
         -0.20241486,  0.20586358, -0.14051712, -0.05082972,  0.09685856,
         -0.09528857, -0.05195801,  0.25050035,  0.04628182,  0.12991883,
          0.3084107 ,  0.19580044, -0.00762283,  0.16998617, -0.083472  ,
          0.05702918, -0.02330957, -0.15591119, -0.15429808, -0.11416874,
          0.16526413,  0.15204333,  0.27533612, -0.01074342, -0.19649354,
          0.00527963,  0.20814532, -0.17622547,  0.00581562, -0.18096776,
          0.07229584,  0.01609052,  0.23837911,  0.11274231, -0.10241145,
          0.21867535,  0.15403013,  0.16937226, -0.12891638, -0.10871136,
         -0.10423722, -0.08433264, -0.03388758,  0.09582229,  0.10162827,
         -0.17948274, -0.01601475, -0.